In [13]:
import os
import pandas as pd
import tqdm

source_path = "casia-fasd/sanity"

rows = []

# Traverse the directory tree
for root, dirs, files in os.walk(source_path):
    # Loop over the files in the current directory
    for name in files:
        # Check if the file has a .png extension
        if name.endswith(".png"):
            # Print the full path of the file
            file_path = os.path.join(root, name)
            # Get only relative path with source path
            file_path = os.path.relpath(file_path, source_path)
            label = int("real" in root)
            rows.append((file_path, label))
            
df = pd.DataFrame(rows, columns=["filepath", "labels"])
df.head()

,filepath,labels
0,fake/19_5_0057_00.png,0
1,fake/19_6_0171_00.png,0
2,fake/17_HR_3_0012_00.png,0
3,fake/12_HR_4_0216_00.png,0
4,fake/8_HR_2_0150_00.png,0


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
train.head()

,filepath,labels
130,casia-fasd/sanity/real/11_1_0144_00.png,1
186,casia-fasd/sanity/real/13_2_0114_00.png,1
166,casia-fasd/sanity/real/6_1_0048_00.png,1
147,casia-fasd/sanity/real/1_HR_1_0246_00.png,1
63,casia-fasd/sanity/fake/6_6_0102_00.png,0


In [10]:
train.to_csv(os.path.join(source_path, "train_split.csv"), index=None)
train.to_csv(os.path.join(source_path, "val_split.csv"), index=None)